In [96]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os

import time

urlList = set()
areaList = set()

basicArea = ['jiangan', 'jianghan', 'qiaokou', 'dongxihu', 'wuchang', 'qingshan', 'hongshan', 'donghugaoxin',
             'jiangxia', 'caidian', 'huangbei', 'xinzhou','zhuankoukaifaqu']
headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWekit 537.36 (KHTML, like Gecko) Chrome",
            "Accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"}

def getSubUrl(basicArea):
    global areaList
    for area in basicArea:
        print('现在进行到' + area)
        url = 'http://wh.lianjia.com/ershoufang/' + area
        areaHtml = urlopen(url)
        bsObj = BeautifulSoup(areaHtml)
        print(url)
        aList = bsObj.select('div[data-role="ershoufang"]')[0].findAll('div')[1].findAll('a')
        for a in aList:
            if a['href'] not in areaList:
                areaList.add(a['href'][12:-1])
    with open('./lianjia/subAreaList', 'a+') as subAreaList:
        for area in areaList:
            subAreaList.write(area + '\n')

def scrapHouse(areaList):
    global urlList
    for area in areaList:
        with open('./lianjia/' + area, 'a+') as txtFile:
            basicUrl = 'https://wh.lianjia.com/ershoufang/' + area
            print('\n\n现在进行到' + area )
            pageNum = getPageNum(basicUrl)
            for num in range(1, pageNum+1):
                url = basicUrl + '/pg' + str(num)
                html = urlopen(url)
                try:
                    html = urlopen(url)
                    bsObj = BeautifulSoup(html)
                    listLen = len(bsObj.select('ul[class^="sell"]')[0].findAll('li'))
                    linkdivs = bsObj.select('div[class=title]')[1:listLen+1]
                    for div in linkdivs:
                        link = div.a['href']
                        if link not in urlList:
                            urlList.add(link)
                            txtFile.write(link + '\n')
                    print('地区:' + area + '第' + str(num) + '页爬取成功\n')
                except:
                    print('地区:' + area + '第' + str(num) + '页爬取失败\n')


def getPageNum(url):
    html = urlopen(url)
    bsObj = BeautifulSoup(html)
    num = eval(bsObj.select('div[page-data]')[0]['page-data'])['totalPage']
    return num

def cleanDataOne(dataList):
    data = list()
    for item in dataList:
        data.append(item[4:])
    return data

def cleanDataTwo(dataList):
    return dataList[1::2]

getSubUrl(basicArea)
scrapHouse(areaList)

现在进行到jiangan
http://wh.lianjia.com/ershoufang/jiangan
现在进行到jianghan
http://wh.lianjia.com/ershoufang/jianghan
现在进行到qiaokou
http://wh.lianjia.com/ershoufang/qiaokou
现在进行到dongxihu
http://wh.lianjia.com/ershoufang/dongxihu
现在进行到wuchang
http://wh.lianjia.com/ershoufang/wuchang
现在进行到qingshan
http://wh.lianjia.com/ershoufang/qingshan
现在进行到hongshan
http://wh.lianjia.com/ershoufang/hongshan
现在进行到donghugaoxin
http://wh.lianjia.com/ershoufang/donghugaoxin
现在进行到jiangxia
http://wh.lianjia.com/ershoufang/jiangxia
现在进行到caidian
http://wh.lianjia.com/ershoufang/caidian
现在进行到huangbei
http://wh.lianjia.com/ershoufang/huangbei
现在进行到xinzhou
http://wh.lianjia.com/ershoufang/xinzhou
现在进行到zhuankoukaifaqu
http://wh.lianjia.com/ershoufang/zhuankoukaifaqu


现在进行到changfengchangmatou
地区:changfengchangmatou第1页爬取成功

地区:changfengchangmatou第2页爬取成功

地区:changfengchangmatou第3页爬取成功

地区:changfengchangmatou第4页爬取成功

地区:changfengchangmatou第5页爬取成功

地区:changfengchangmatou第6页爬取成功

地区:changfengchangmatou第7页爬取成功

地区:changfengchan

In [16]:
import csv
import codecs
from bs4 import BeautifulSoup
import os
from urllib.request import urlopen
import re
headList = ['房屋类型', '所在楼层', '建筑面积', '户型结构', '套内面积', '建筑类型', '房屋朝向', '建筑结构', '装修情况', '梯户比例', '是否配备电梯', '产权年限', '挂牌时间', '交易权属', '上次交易', '房屋用途', '是否满两年', '产权所属(是否共有)', '抵押信息', '房本备件', '小区名', '首付', '税费', '挂牌价', '单价','url']
with codecs.open('./lianjia.csv', 'w+', 'utf-8') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(headList)
    with open('./lianjia/baibuting', 'r') as urlFile:
        urlList = urlFile.read().split('\n')
    for url in urlList:
        print('正在进行' + url)
        html = urlopen(url)
        bsObj = BeautifulSoup(html)
        writer.writerow(cleanData(bsObj, url))
        print('url:' + url + '成功')

正在进行https://wh.lianjia.com/ershoufang/104101233595.html
url:https://wh.lianjia.com/ershoufang/104101233595.html成功
正在进行https://wh.lianjia.com/ershoufang/104101085322.html
url:https://wh.lianjia.com/ershoufang/104101085322.html成功
正在进行https://wh.lianjia.com/ershoufang/104101197835.html
url:https://wh.lianjia.com/ershoufang/104101197835.html成功
正在进行https://wh.lianjia.com/ershoufang/104100931271.html
url:https://wh.lianjia.com/ershoufang/104100931271.html成功
正在进行https://wh.lianjia.com/ershoufang/104101071586.html
url:https://wh.lianjia.com/ershoufang/104101071586.html成功
正在进行https://wh.lianjia.com/ershoufang/104100910730.html
url:https://wh.lianjia.com/ershoufang/104100910730.html成功
正在进行https://wh.lianjia.com/ershoufang/104101124221.html
url:https://wh.lianjia.com/ershoufang/104101124221.html成功
正在进行https://wh.lianjia.com/ershoufang/104100642457.html
url:https://wh.lianjia.com/ershoufang/104100642457.html成功
正在进行https://wh.lianjia.com/ershoufang/104101021625.html
url:https://wh.lianjia.com/ersho

HTTPError: HTTP Error 404: Not Found

In [15]:
# 数据整理

def cleanData(bsObj, url):
    data = list()
    # 基本信息
    a = bsObj.select_one('div[class="base"] div[class="content"]').get_text().replace(' ', '').replace('\n\n', '').split('\n')
    # 交易属性
    b = bsObj.select_one('div[class="transaction"] div[class="content"]').get_text().replace(' ', '').replace('\n', ' ').replace('  ', ' ').replace('  ', ' ').split(' ')[1:-1][1::2]   
    # 小区位置
    c = bsObj.select_one('div[class="communityName"] a').get_text()
    # 税费
    d = bsObj.select_one('span[class="taxtext"]').get_text()
    # 总价
    e = bsObj.select_one('span[class="total"]').get_text()
    # 单价
    f = bsObj.select_one('span[class="unitPriceValue"]').get_text()
    for num in range(0, len(a)):
        if num == 2 or num == 4 or num == 11:
            data += re.findall('[0-9.]+', a[num])
        elif num == 10:
            if a[num] == '有':
                data.append(True)
            else:
                data.append(False)
        else:
            data.append(a[num][4:])
    if b[4] == '未满两年':
        b[4] = False
    else:
        b[4] = True
    if b[5] == '非共有':
        b[5] = False
    else:
        b[5] = True
    data += b
    data.append(c)
    data += re.findall('[0-9.]+',d)
    data += re.findall('[0-9.]+',e)
    data += re.findall('[0-9.]+',f)   
    numlist = [2,4,11,21,22,23,24]
    #for num in numlist:
    #    data[num] = float(data[num])
    data.append(url)
    return data
# 房屋类型, 所在楼层, 建筑面积, 户型结构, 套内面积, 建筑类型, 房屋朝向, 建筑结构, 装修情况, 梯户比例, 是否配备电梯, 产权年限, 挂牌时间, 交易权属, 上次交易, 房屋用途, 是否满两年, 产权所属(是否共有), 抵押信息, 房本备件, 小区名, 首付, 税费, 挂牌价, 单价      

In [182]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

mm = '首付65.1万 税费13.1万(仅供参考)'
ee = '22735元/平米'
re.findall("[0-9.]+", mm)

['65.1', '13.1']

In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import datetime
import random

random.seed(datetime.datetime.now())
def getLinks(articleUrl):
    html = urlopen('http://en.wikipedia.org' + articleUrl)
    bsObj = BeautifulSoup(html)
    return bsObj.find('div', {'id': 'bodyContent'}).findAll("a", href=re.compile('^(/wiki/)((?!:).)*$'))
links = getLinks("/wiki/Kevin_Bacon")
'''while len(links) > 0:
    newArticle = links[random.randint(0, len(links) - 1)].attrs['href']
    print((newArticle))
    links = getLinks(newArticle)'''

"while len(links) > 0:\n    newArticle = links[random.randint(0, len(links) - 1)].attrs['href']\n    print((newArticle))\n    links = getLinks(newArticle)"